<a href="https://colab.research.google.com/github/Davo00/INLTP-Assignments/blob/main/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Heidelberg University**

**Data Science  Group**
    
Prof. Dr. Michael Gertz  

Ashish Chouhan, Satya Almasian, John Ziegler, Jayson Salazar, Nicolas Reuter
    
January 16, 2024
    
Natural Language Processing with Transformers

Winter Semster 2023/2024     
***

# **Assignment 4: Question Answering**
**Due**: Monday, January 29, 2024, 2pm, via [Moodle](https://moodle.uni-heidelberg.de/course/view.php?id=19251)



### **Submission Guidelines**

- Solutions need to be uploaded as a **single** Jupyter notebook. You will find several pre-filled code segments in the notebook, your task is to fill in the missing cells.
- For the written solution, use LaTeX in markdown inside the same notebook. Do **not** hand in a separate file for it.
- Download the .zip file containing the dataset but do **not** upload it with your solution.
- It is sufficient if one person per group uploads the solution to Moodle, but make sure that the full names of all team members are given in the notebook.

***

## **Task 1: Retrieval Augmented Generation (RAG)** ( 4.5 + 3 + 4 + 3 + 1.5 = 16 points)

In this task, we look at using the open source `Llama-13b-chat` model for creating a RAG system. You must first apply for access to Llama 2 models via [this](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) form (access is typically granted within a few hours). etrieval augmented generation you also need to request to use the model on Hugging Face by going to the [model](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) card. ***Note that the emails you provide for your Hugging Face account must match the email you used to request Llama 2.***

The final piece that you need is a Hugging Face authentication token. You can find such a token by going to the `setting` in your Hugging Face profile, under the `Access Token` menu you can generate a new token.

To store the document you will need a free Pinecone [API key](https://app.pinecone.io/).
Make sure you have these pieces ready before starting to work on this task.

----
When ready, let's start by downloading the necessary packages.

It is advised to proceed with this notebook with a GPU (if you are on Colab make sure that a GPU environment is activated.)


Place all the access tokens in the `.env` file and upload it to the working directory (if you are running this notebook locally, you can change the path to fit your working directory). Please use the following format:


```
HF_AUTH= "Hugging Face Authentication Key"
PINECONE_API_KEY="Pincone API Key"
PINECONE_ENVIRONMENT="Pinecone Environment"
```

Run the cell below to load the access tokens into the environment variables.

In [1]:
%pip -q install python-dotenv

In [2]:

%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install -q transformers
%pip install -q sentence-transformers
%pip install -q pinecone-client
%pip install -q datasets
%pip install -q accelerate
%pip install -q einops
%pip install -q langchain
%pip install -q xformers
%pip install -q bitsandbytes
%pip install -q matplotlib seaborn tqdm
%pip install -q chromadb
%pip install -q evaluate
%pip install -q rouge_score
%pip install -q bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensio

In [2]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



## Subtask 1.1: Data Preparation



We need a collection of documents to perform our retrieval on. To make it closer to your final project, you will be downloading and using a subset of the LangChain documentation. We get some of the `.html` files located on the site. The code below will download all HTML files from the links on the webpage into a `docs` directory. `-l1` limits the download to only the first level of depth.


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!wget -r -l1 -A.html -P docs https://api.python.langchain.com/en/stable/langchain_api_reference.html

--2024-01-28 10:03:14--  https://api.python.langchain.com/en/stable/langchain_api_reference.html
Resolving api.python.langchain.com (api.python.langchain.com)... 104.17.32.82, 104.17.33.82, 2606:4700::6811:2152, ...
Connecting to api.python.langchain.com (api.python.langchain.com)|104.17.32.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘docs/api.python.langchain.com/en/stable/langchain_api_reference.html’

api.python.langchai     [ <=>                ] 253.61K  --.-KB/s    in 0.03s   

2024-01-28 10:03:15 (7.83 MB/s) - ‘docs/api.python.langchain.com/en/stable/langchain_api_reference.html’ saved [259698]

Loading robots.txt; please ignore errors.
--2024-01-28 10:03:15--  https://api.python.langchain.com/robots.txt
Reusing existing connection to api.python.langchain.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 130 [text/plain]
Saving to: ‘docs/api.python.langchain.com/robots.txt.tmp’

api.python.langchai

 The docs are going to be used as input text for answering questions that a normal language model might not be aware of (LangChain docs is not necessarily part of its training data of Llama2). We can use LangChain itself to process these docs. Use the [ReadTheDocsLoader](https://python.langchain.com/docs/integrations/document_loaders/readthedocs_documentation) to load the docs from the `docs` folder.

 At the time of creating this notebook, there  `423` documents were downloaded. However, since the documentation is being updated regularly this number might be different for you.

In [5]:
from langchain.document_loaders import ReadTheDocsLoader
#### your code ####
loader = ReadTheDocsLoader("docs")
docs = loader.load()
#### your code ####
len(docs)

418

Let's take a look at one of the documents. You see that LangChain has created a `Document` object. Look at the example below and fill in the cells to print out the text content and URL of the page (the URL of the page should starts with `https://`).

In [6]:
docs[10]

Document(page_content='langchain_anthropic 0.0.1.post2¶\nlangchain_anthropic.chat_models¶\nClasses¶\nchat_models.ChatAnthropicMessages\nBeta ChatAnthropicMessages chat model.\nFunctions¶', metadata={'source': 'docs/api.python.langchain.com/en/stable/anthropic_api_reference.html'})

In [7]:
#### your code ####
## For all the documents, the colab complains that the IO rate exceeded, ehcne only one doc will be analyzed!
page_content= [doc.page_content for doc in docs]
page_url= ["https://"+doc.metadata["source"] for doc in docs]


In [8]:
doc = docs[10]
page_content= doc.page_content
page_url= "https://"+doc.metadata["source"]
#### your code ####
print(page_content)
print(page_url)

langchain_anthropic 0.0.1.post2¶
langchain_anthropic.chat_models¶
Classes¶
chat_models.ChatAnthropicMessages
Beta ChatAnthropicMessages chat model.
Functions¶
https://docs/api.python.langchain.com/en/stable/anthropic_api_reference.html


As you can imagine the documents can be long and if multiple of them are required as context to answer questions, we need to take the document lengths into account.
This is due to the fact that language models do not have unlimited context span. In our case, we plan to use Llama2 for this project, where the maximum token limit is 4096. This limit is not only the input but also takes the generated output into account, moreover, you need to leave room for the query and instructions as well. Therefore, it is important to chunk the longer documents into smaller-sized fragments.

Based on your use case and how many contexts you plan to feed into the model the length of these fragments will differ.
In this case, we choose to assign 2000 tokens to context and choose to generate the answer from 5 context fragments, which leaves us with 400 tokens per context fragment as the maximum chunk size.

To count the number of tokens in a chunk, we need to load the correct tokenizer for Llama2. Fill the code cell below to load the correct tokenizer and use it to complete the function that counts the number of tokens per given chunk.

**Hint:** you need to use your Hugging Face authentication token to load the tokenizer.

In [9]:
#If you get an error here during the first import from the `transformers` package, restart the kernel and try again.
#### your code ####
import os
from transformers import LlamaForCausalLM, LlamaTokenizer
hf_auth = os.getenv("HF_AUTH")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",use_auth_token=hf_auth)
#### your code ####

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1895: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [10]:
def token_len(text):
  #### your code ####
    tokens = tokenizer(text)['input_ids']
    # Count the number of tokens
    num_tokens = len(tokens)
    return num_tokens
    #### your code ####

Count the number of tokens for all documents and use it to compute minimum, maximum, and average token count statistics across all documents. Depending on how the documentation is updated by the time you run the cell below the numbers might slightly differ.

In [11]:
#### your code ####
token_counts = [ token_len(doc.page_content) for doc in docs]
min_tokens= min(token_counts)
avg_tokens= sum(token_counts) / len(token_counts)
max_tokens= max(token_counts)
#### your code ####
print(f"""Min: {min_tokens}
Avg: {avg_tokens}
Max: {max_tokens}""")

Min: 49
Avg: 4084.94019138756
Max: 38897


Now we will use LangChain's built-in chunking functionality to split the text into smaller chunks. LangChain offers a variety of text splitters that you can check out [here](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.text_splitter).
Use the general-purpose splitter that splits text by recursively looking at characters. Use this class to split the text into 400 token-sized chunks, where the length of each chunk is computed based on the `token_len` function. The length is not the only criterion for splitting, if any of these separators `'\n\n', '\n', ' ', ''` is encountered, we will have a new chunk.
Since splitting only based on maximum length might result in incoherent chunks for every consecutive chunk, let the chunk overlap by 50 tokens. This way,  we preserve some of the previous context while chunking.

In [12]:
#### your code ####
from langchain.text_splitter import RecursiveCharacterTextSplitter

separators = ['\n\n', '\n', ' ', '']
chunk_size = 400
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, length_function=token_len, separators=separators, chunk_overlap=50)

#### your code ####

In [13]:
chunks = text_splitter.split_text(docs[100].page_content)
len(chunks)

41

In [14]:
token_len(chunks[0])

330

The next step is to apply the splitting function to all the documents in our corpus and to save our chunks in a logical way. We also want to assign a unique ID to each chunk so we know which part of the documentation they come from. In the end, the corpus should be transformed into a list of dictionaries of the following format:


```
[
    {
        "id": "glossary-0",
        "text": "first chunk of the document glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    },
    {
        "id": "glossary-1",
        "text": "second chunk of glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    }
    ...
]
```

Construct the IDs by taking the name of the page before the suffix `.html` and appending a chronological number indicating which chunk it is.


In [15]:
from tqdm.auto import tqdm

documents = []

for doc in tqdm(docs):
  #### your code ####
    url = "https://"+doc.metadata["source"]
    uid = url.split("/")[-1].replace(".html", "")
    chunks = text_splitter.split_text(doc.page_content)

    count = 0
    for chunk in chunks:
      documents.append({
          "id": uid+"-"+str(count),
          "text": chunk,
          "source": url
      })
      count += 1
  #### your code ####
len(documents) # once again this value might differ based on how the LangChain documentation is updated

  0%|          | 0/418 [00:00<?, ?it/s]

6065

For the next steps, we require a `DataFrame`.

In [16]:
import pandas as pd
data = pd.DataFrame(documents)
data.head()

,id,text,source
0,mistralai_api_reference-0,langchain_mistralai 0.0.3¶\nlangchain_mistrala...,https://docs/api.python.langchain.com/en/stabl...
1,google_genai_api_reference-0,langchain_google_genai 0.0.6.post1¶\nlangchain...,https://docs/api.python.langchain.com/en/stabl...
2,core_api_reference-0,langchain_core 0.1.16¶\nlangchain_core.agents¶...,https://docs/api.python.langchain.com/en/stabl...
3,core_api_reference-1,Patch a runnable config with context getters a...,https://docs/api.python.langchain.com/en/stabl...
4,core_api_reference-2,Async callback manager that handles callbacks ...,https://docs/api.python.langchain.com/en/stabl...


#### ${\color{red}{Comments\ 1.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.2: Document Embedding Pipeline


In this task, we initialize the embedding pipeline to transform the chunks into vector embeddings using Hugging Face and LangChain. These embeddings are used for similarity search between the query and the chunks to retrieve the most relevant chunks.
  We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding, which is a rather small model that you can easily run on Colab. Initialize the model using `HuggingFaceEmbeddings` to use Hugging Face via Langchain. The encoding batch size should be 32, and make sure that the model is placed on the correct device, otherwise, this can take a long time.

In [17]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
import pinecone
from tqdm.notebook import tqdm

In [18]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0' if cuda.is_available() else 'cpu'
docs = [
    "An example document",
    "A second document as an example"
]
### your code ###
import requests

embeddings_pipeline = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs={"device": device},encode_kwargs={"batch_size": 32})
### your code ###

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Embed the example documents using the model you created and check the output.
The output should be a list of lists, containing the embeddings.

In [19]:
### your code ###
embeddings = [embeddings_pipeline.embed_query(doc) for doc in docs]
### your code ###
print("number of docs:",len(embeddings))
print("dimension of docs:",len(embeddings[0]))

number of docs: 2
dimension of docs: 384


Now we use the embedding pipeline created above to store the embeddings in a Pinecone vector index. First, lets setup the Pinecone environment, collect your API key and environment name from the environment variables, and initiate Pinecone with them.

In [20]:
### your code ###
pinecone_api_key=os.getenv("PINECONE_API_KEY")
pinecone_environment=os.getenv("PINECONE_ENVIRONMENT")
from pinecone import Pinecone, PodSpec

# Initialize Pinecone with API key and environment
pc=Pinecone(api_key=pinecone_api_key, environment=pinecone_environment)
### your code ###

Initialize the index `rag-assignment` inside Pinecone. Use the cosine similarity as similarity metric. Keep in mind that if you run this multiple times on a free tier, where only one index is allowed, you need to remove the index created to make room for a new one (Pinecone index gets archived automatically after 14 days of inactivity).

In [21]:
def create_index(index_name, dimension, metric):
  pc.create_index(
    name=index_name,
    dimension=dimension,
    metric=metric,
    spec=PodSpec(
      environment="gcp-starter"
    )
  )

index_name = 'rag-assignment'
### your code ###
if pc.list_indexes() != []:
  pc.delete_index(index_name)

create_index(index_name, 384, "cosine")
### your code ###

Lets take a look at the index you created. As of now the index should be empty but have the correct embedding dimension.

In [22]:
index_name = 'rag-assignment'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Process the dataset in batches of `32` and push the vectors to the Pinecone index. Your index should include the IDs and embeddings for each chunk. As metadata, pass the original text as `text` and the URL as `source` (no need to add the `https`). We use this metadata later to retrieve the original text.

In [23]:
batch_size = 32
for i in tqdm(range(0, len(data), batch_size)):
  ### your code ###
    items = []
    batch = data.iloc[i:i + batch_size]
    #print(type(batch))
    for x in range(len(batch)):
      item = batch.iloc[x]
      id = item["id"]
      text = item["text"]
      embeds = embeddings_pipeline.embed_query(text)
      source =  item["source"].replace("https://", "")

      # TODO: Check if this format is correct ? Or should each chunk be saved on its own and not in a batch ?
      items.append({
          "id": id,
          "values": embeds,
          "metadata": {
              "text": text,
              "source": source
          }
      })
    index.upsert(vectors=items)
    ### your code ###


  0%|          | 0/190 [00:00<?, ?it/s]

Now if we look at the index statistics we should have vectors of dimension `384`.

In [24]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.05952,
 'namespaces': {'': {'vector_count': 5952}},
 'total_vector_count': 5952}

#### ${\color{red}{Comments\ 1.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.3: Text Generation Pipeline


So far we have our index ready and a way to find the most similar chunks to our query. Now, we need a way to generate the answer from the retrieved chunks. For this purpose, we use the `text-generation` pipeline from Hugging Face (refer to the Hugging Face [tutorial](https://moodle.uni-heidelberg.de/pluginfile.php/1286642/mod_resource/content/1/HuggingFace.ipynb)) and load it into LangChain using a wrapper.

In [25]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'meta-llama/Llama-2-13b-chat-hf'

hf_auth = os.getenv("HF_AUTH")

Quantization techniques reduce memory and computational costs by representing weights and activations with lower-precision data types like 8-bit integers (int8). This enables loading larger models you normally wouldn’t be able to fit into memory, and thus speeds up inference.
To make the process of model quantization more accessible, Hugging Face has seamlessly integrated with the [Bitsandbytes](https://huggingface.co/docs/accelerate/usage_guides/quantization) library.

Define a config from `Bitsandbytes` that enables 4-bit quantization and set the nested quantization to `true`. This changes the datatype from float 32 (default) to normalized float 4 datatype to contain 4 bits of information.
Additionally, add a compute type to store weights in 4-bits, but the computation to happen in 16-bit (bfloat16).
Moreover, set the `bnb_4bit_use_double_quant` to true, which uses a second quantization after the first one to save an additional 0.4 bits per parameter.
Refer to [here](https://huggingface.co/docs/transformers/main_classes/quantization) for more information.

In [26]:
  ### your code ###
import torch

bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    nested_quantization=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True)
  ### your code ###

Use your Hugging Face token to load the correct model configuration using the `transformers` library.

In [27]:
### your code ###

model_config = transformers.AutoConfig.from_pretrained(model_id, token=hf_auth)
### your code ###


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [28]:
bitsAndBites_config

BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_type": "fp4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

Load the model for text generation (pay attention to the model type) using the configuration file you have defined, with the specified quantization, and set the `trust_remote_code` flag to `true`. Another flag that is useful for large mode is  `device_map="auto"`. By setting this flag, Accelerate will determine where to put each layer to maximize the use of GPUs and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM).

It will take a while for the model to download.

In [29]:
#Loading the model will take some time, (roughly 5 min)
### your code ###
model = transformers.AutoModelForCausalLM.from_pretrained(model_id, config=model_config,
                                                          quantization_config=bitsAndBites_config,
                                                          device_map="auto",
                                                          trust_remote_code=True,
                                                          token=hf_auth)
### your code ###
model.eval()# we only use the model for inference
print(f"Model loaded ")

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded 


You can even check the memory footprint of your model using the `get_memory_footprint` method.


In [30]:
model.get_memory_footprint() / 1e9

7.08397056

The next thing we need to do is initialize a `text-generation` pipeline with Hugging Face that uses the Llama2 model to generate some text, given some input. We will then use this pipeline inside LangChain to build our question-answering system.
`text-generation` pipeline generates text from a language model conditioned on a given input. The pipeline is similar to other Hugging Face pipelines and requires two things that we must initialize:

1.   A language model, in this case, it will be `meta-llama/Llama-2-13b-chat-hf`.
2.   A tokenizer for the language model.

LangChain expects the full-text outputs, therefore set the `return_full_text` to true. You can also pass additional generation parameters to the model.
Since we want the questions to be answered mainly based on the retrieved chunks, let's set the model temperature to a low value of 0.01 to reduce randomness. Additionally, add a repetition penalty of 1.1 to stop the model from repeating itself and the maximum number of generation tokens to 512.

In [31]:
### your code ###
import torch
from transformers import pipeline, AutoTokenizer

text_generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    torch_dtype=torch.float16,
    tokenizer=tokenizer,
    return_full_text=True,
    temperature=0.01,
    max_new_tokens=512,
    repetition_penalty=1.1)

generate_text = ""
### your code ###

We provide the language model a general question to make sure our pipeline is working correctly.

In [32]:
sample_input="Explain to me the difference between alligator and crocodile."
### your code ###

generated_text=text_generation_pipeline(sample_input)
### your code ###
print(generated_text)

[{'generated_text': 'Explain to me the difference between alligator and crocodile.\n\nAlligators and crocodiles are both large, carnivorous reptiles that live in aquatic environments, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape, while crocodiles are more streamlined.\n2. Habitat: Alligators are found only in freshwater environments, such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Nesting habits: Alligators build mounds of vegetation and mud to lay their eggs, while crocodiles dig holes in the sand or mud to lay the

Use the LangChain Hugging Face wrapper, as subset of [LLM chain](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) to create an interface for the text generation pipeline.

In [33]:
### your code ###
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
### your code ###

To confirm that it works the same way, use the sample input to generate text using the llm chain. The input should be passed as the `prompt` to the language model.

In [34]:
### your code ###
llm.invoke(sample_input)
### your code ###

'\n\nAlligators and crocodiles are both large, carnivorous reptiles that live in aquatic environments, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape, while crocodiles are more streamlined.\n2. Habitat: Alligators are found only in freshwater environments, such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Nesting habits: Alligators build mounds of vegetation and mud to lay their eggs, while crocodiles dig holes in the sand or mud to lay their eggs.\n5. Jaw structure: Alligators have a different jaw structure than crocod

#### ${\color{red}{Comments\ 1.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.4: Question Answering Chain


For Retrieval Augmented Generation (RAG) in LangChain, we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object.

`RetrievalQA` is a method for question-answering tasks, utilizing an index to retrieve relevant documents or text chunks, it is suitable for straightforward Q&A applications.

`RetrievalQAWithSourcesChain` is an extension of RetrievalQA that chains together multiple sources of information, providing context and the source for answers.

 For both of these, we need an LLM and a Pinecone index. For LangChain to be able to use the Pinecone index, we need to initialize it through the LangChain vector store.

 **Hint**: You need to explicitly tell the vector storage where to find the original text.

In [35]:
from langchain.vectorstores import Pinecone
### your code ###

vectorstore = Pinecone(index, embeddings_pipeline, "text")

### your code ###

Let's try a query that is specific to the LangChain documentation and see which chunks are relevant. Use the vector storage defined above to find the top-3 chunks related to the given query.

In [36]:
query = 'what is a LangChain Agent?'
### your code ###
top_chunks = vectorstore.similarity_search(query, k=3)
top_chunks
### your code ###

[Document(page_content='langchain 0.1.4¶\nlangchain.agents¶\nAgent is a class that uses an LLM to choose a sequence of actions to take.\nIn Chains, a sequence of actions is hardcoded. In Agents,\na language model is used as a reasoning engine to determine which actions\nto take and in which order.\nAgents select and use Tools and Toolkits for actions.\nClass hierarchy:\nBaseSingleActionAgent --> LLMSingleActionAgent\n                          OpenAIFunctionsAgent\n                          XMLAgent\n                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent\nBaseMultiActionAgent  --> OpenAIMultiFunctionsAgent\nMain helpers:\nAgentType, AgentExecutor, AgentOutputParser, AgentExecutorIterator,\nAgentAction, AgentFinish\nClasses¶\nagents.agent.Agent\n[Deprecated]  Agent that calls the language model and deciding the action.\nagents.agent.AgentExecutor\nAgent that is using tools.\nagents.agent.AgentOutputParser\nBase class for parsing agent output into agent acti

Now use the `vectorstore` and `llm` to initialize the `RetrievalQA` object, which showcases question answering over an index. `RetrievalQA` is a document chain, these are useful for summarizing documents, answering questions about documents, extracting information from documents, and more. All such chains operate with 4 different chain types:


1.   `stuff`: it takes a list of documents, inserts them all into a prompt, and passes that prompt to an LLM.
2.   `refine`: it constructs a response by looping over the input documents and iteratively updating its answer. It is well-suited for tasks that require analyzing more documents than can fit in the model’s context.
3. `map_reduce`:  it first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combined documents chain to get a single output (the Reduce step).
4. `map_re_rank`: it runs an initial prompt on each document that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest-scoring response is returned.

For this assignment, we focus only on the first type. Make sure to set the `verbose` to `true`, so we can see the different stages of processing that happens while answering a question (you might need to set this parameter more than once). As mentioned before, we want our retrieve to input top-5 most similiar chunks to the query to generate an answer.

In [37]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

### your code ###
retriever = VectorStoreRetriever(vectorstore=vectorstore, search_kwargs={"k": 5})
rag_pipeline = RetrievalQA.from_llm(llm=llm, retriever=retriever, verbose=True)

### your code ###
query='what is a LangChain Agent?'


First, we try to answer the question only using Llama2. As you see the answer is not convincing as it does not have access to the LangChain documentation.

In [38]:
llm(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nA LangChain Agent is an AI-powered chatbot that uses natural language processing (NLP) to understand and respond to user queries. It is designed to provide personalized support and answer questions in real-time, 24/7. The agent is trained on a large dataset of customer interactions, which enables it to understand the nuances of human language and provide accurate responses.\n\nLangChain Agents are powered by advanced machine learning algorithms that allow them to learn from each interaction and improve their performance over time. They can be integrated with various messaging platforms such as Facebook Messenger, WhatsApp, Slack, and more. This allows businesses to provide seamless support to their customers across multiple channels.\n\nSome of the key features of LangChain Agents include:\n\n1. Natural Language Processing (NLP): LangChain Agents use NLP to understand and interpret user queries, allowing them to provide accurate responses.\n2. Machine Learning: LangChain Agents ar

Now use the Pipeline from above and see how the answer changes.

In [39]:
### your code ###
rag_pipeline(query)
### your code ###


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is a LangChain Agent?',
 'result': ' A LangChain Agent is a piece of software that uses machine learning to perform tasks. It is built on top of the LangChain framework, which provides a set of tools and libraries for building and training machine learning models. The agent is trained on a specific task, such as answering questions or generating text, and it uses a combination of natural language processing (NLP) and machine learning algorithms to perform the task. The agent can be controlled using a simple API, and it can be integrated into a variety of applications and systems.'}

#### ${\color{red}{Comments\ 1.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.5: Conversational Retrieval Chain




We can also extend our retrieval chain to be able to remember the previous questions and answer the current question by looking at the previous context.
The important part of a conversational model is conversation memory, which transforms the stateless language model to be able to remember previous interactions, e.g., similiar to ChatGPT. In this subtask, we will use LangChain to create a conversational memory.


To implement the memory we use `ConversationalRetrievalChain`.
This chain takes in chat history (a list of messages) and new questions and then returns an answer to that question. The algorithm for this chain consists of three parts:

1. Use the chat history and the new question to create a new question that contains the information from the previous context.

2. This new question is passed to the retriever and relevant documents are returned.

3. The retrieved documents are passed to an LLM to generate a final response.

In [40]:
from langchain.chains import ConversationalRetrievalChain
chat_history = []

### your code ###
qa_conversation = ConversationalRetrievalChain.from_llm(llm, retriever, verbose=True, chain_type="stuff")
result = qa_conversation({"chat_history": chat_history, "question": query})
### your code ###




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

langchain 0.1.4¶
langchain.agents¶
Agent is a class that uses an LLM to choose a sequence of actions to take.
In Chains, a sequence of actions is hardcoded. In Agents,
a language model is used as a reasoning engine to determine which actions
to take and in which order.
Agents select and use Tools and Toolkits for actions.
Class hierarchy:
BaseSingleActionAgent --> LLMSingleActionAgent
                          OpenAIFunctionsAgent
                          XMLAgent
                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent
BaseMultiActionAgent  --> OpenAIMultiFunctionsAgent
Main helpers:
AgentType, AgentExecutor, AgentOutputParser, AgentExecutorIterator,
AgentAction, AgentFinish
Classes¶
agents

In [41]:
result["answer"]

' A LangChain Agent is a piece of software that uses machine learning models to perform tasks. It is built on top of the LangChain framework and can be used to perform various tasks such as answering questions, generating text, and more.\n\nPlease let me know if you need any further information or clarification.'

Change the chat history to contain the previous question and answer pair and ask a follow-up question.  

In [42]:
follow_up="What are tools and toolkits?"

### your code ###
chat_history.append((query, result["answer"]))
result = qa_conversation({"chat_history": chat_history, "question": follow_up})
### your code ###



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: what is a LangChain Agent?
Assistant:  A LangChain Agent is a piece of software that uses machine learning models to perform tasks. It is built on top of the LangChain framework and can be used to perform various tasks such as answering questions, generating text, and more.

Please let me know if you need any further information or clarification.
Follow Up Input: What are tools and toolkits?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

[Deprecated]  Chat Agent.[Deprecated] Chat Agent.
agents.chat.outp

This is the previous context that was fed in alongside the new question.

In [43]:
chat_history

[('what is a LangChain Agent?',
  ' A LangChain Agent is a piece of software that uses machine learning models to perform tasks. It is built on top of the LangChain framework and can be used to perform various tasks such as answering questions, generating text, and more.\n\nPlease let me know if you need any further information or clarification.')]

The current question is answered by knowing that the tools and toolkits are referring to a LangChain Agent, which was part of the previous question.

In [44]:
result['answer']

'  Sure! Toolkits are collections of tools that can be used together to accomplish a specific task or set of tasks. They are similar to toolboxes, but instead of containing physical tools, they contain software tools that can be used to perform various functions.\n\nIn contrast, tools are individual pieces of software that can be used to perform a specific function or set of functions. Tools are often designed to be used independently, while toolkits are designed to be used together to achieve a common goal.\n\nFor example, the AIN Network toolkit contains several tools that can be used together to perform various functions related to artificial intelligence, such as natural language processing and machine learning. Each tool in the toolkit serves a specific purpose, and they can be combined in different ways to achieve different goals.\n\nSo, in summary, toolkits are collections of tools that can be used together to accomplish a specific task or set of tasks, while tools are individua

#### ${\color{red}{Comments\ 1.5}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 2: Advanced RAG Techniques and Evaluation (4 + 5 = 9 points)**

Now that you have successfully implemented your first RAG system, we dive into more advanced techniques and learn how to evaluate your methods using metrics you learned during the lecture. We focus on evaluation with an already annotated dataset. To this end, we load a small subset of [NarrativeQA](https://huggingface.co/datasets/narrativeqa), which is an English-language dataset of stories and corresponding questions designed to test reading comprehension, especially on long documents. We only load 30 samples from the data, as you will see in the upcoming sections, answer generation takes quite some time. In actual setting, it is advised to use a much larger set to obtain statistically significant results.

In [45]:
from datasets import load_dataset
dataset = load_dataset("satyaalmasian/narrativeqa_subset",split="train[:30]")
len(dataset)

Generating train split:   0%|          | 0/317 [00:00<?, ? examples/s]

30

Since we already used our free index in Pinecone for the previous task, we use Chroma, an open-source vector database, instead. As opposed to Pinecone, Chroma creates a collection on your machine.

In [46]:
from langchain.docstore.document import Document
documents=[ doc["text"] for doc in dataset["document"]]
questions=[quest for quest in dataset["question"]]
answers=[ans for ans in dataset["answers"]]
documents=list(set(documents))

In [47]:
docs= [Document(page_content=doc, metadata={"source": "local"}) for doc in documents]

The number of documents is smaller  than the number of questions and answers and each document is used as a reference for multiple questions:

In [48]:
print(len(docs))
print(len(questions))

2
30


## Subtask 2.1: Build Contextual Compression in LangChain

Let's split our documents using the TextSplitter from Task 1 and embed them inside the Chroma database with the embedding model of the previous task.

In [49]:
### your code ###
all_splits = []
for doc in docs:
  print(type(doc))
  break
  splits = text_splitter.split_text(doc)
  all_splits.extend(splits)
### your code ###

<class 'langchain_core.documents.base.Document'>


In [50]:
my_splits = text_splitter.split_documents(docs)

In [51]:
from langchain.vectorstores import Chroma
### your code ###
vectordb = Chroma.from_documents(my_splits, embeddings_pipeline)
retriever = VectorStoreRetriever(vectorstore=vectordb)
### your code ###

In [52]:
print("Fist question in the set:",questions[2]['text'])
r_docs = retriever.get_relevant_documents(questions[2]['text'])
r_docs

Fist question in the set: Why do more students tune into Mark's show?


[Document(page_content="Reporter #2 - Are you on drugs?\n\nPaige - Arrrgh. Talk Hard. Arrrrrgh.\n\nMark - I've got a lot of homework I'm gonna take off alright.\n\nMarla - Mark I know why your really going home. It's because you wanna listen to that \nshow tonight don't you?\n\n<Play Peter Murphy>\n\n<Nora goes to Marks house where she finds him burning his Happy Harry Hardon \nletters>\n\nNora - Hi! What are you doing? You having fun?\n\nMark - Yeah.\n\nNora - Hey, look I took some of these off the wall for you. I mistakingly thought you \nmight want them.\n\nMark - Thanks.\n\nNora - So I guess you're not going on tonight.\n\nMark - Brilliant.\n\nNora - Is this all just a game to you. You know you can't just shout fire in a theatre and \nwalk out. You have a responsibility for the people who believe in you. What is this? \nC'mon say something, say anything. Open your mouth and say get the hell out of here \nbitch.\n\nMark - I can't.\n\nNora - You can't what?\n\nMark - I can't talk.\n\

First, make a simple RAG pipeline that works on top of the Chroma retriever. This retriever should be similar to the previous task. However, since we want to use it for a large number of questions, remove the `verbose` parameters.

In [53]:
from langchain.chains import RetrievalQA
### your code ###
rag_simple = RetrievalQA.from_llm(llm=llm, retriever=retriever, verbose=False)
### your code ###

We look at an example question and compare the answer by RAG to the gold answer from the dataset. Note that the answers can contain multiple lines.

In [54]:
rag_simple(questions[2]['text']) #ignore the warning

{'query': "Why do more students tune into Mark's show?",
 'result': " Because he talks about real issues that matter to them, like drug use and the school's lack of support for students struggling with these issues.\n\nUnhelpful Answer: Because he's cute and has a cool accent."}

In [55]:
answers[2]

[{'text': 'Mark talks about what goes on at school and in the community.',
  'tokens': ['Mark',
   'talks',
   'about',
   'what',
   'goes',
   'on',
   'at',
   'school',
   'and',
   'in',
   'the',
   'community',
   '.']},
 {'text': 'Because he has a thing to say about what is happening at his school and the community.',
  'tokens': ['Because',
   'he',
   'has',
   'a',
   'thing',
   'to',
   'say',
   'about',
   'what',
   'is',
   'happening',
   'at',
   'his',
   'school',
   'and',
   'the',
   'community',
   '.']}]

Apply the `rag_simple` pipeline to all the question in your corpus and accumulate the answers. **It should take around 10 minutes on a T4 GPU on Colab**.

In [56]:
simple_answers=[]
### your code ###
for question in tqdm(questions):
  simple_answers.append(rag_simple(question['text']))
### your code ###

  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

Libraries such as LangChain and [Llamaindex](https://www.llamaindex.ai/) provide a variety of retrieval strategies for building a RAG system. In this subtask, you will use one of these variations called **contextual compression**. This method aims to extract only the relevant information from documents, reducing the need for expensive language model calls and improving response quality. Contextual compression consists of two parts:


1.  **Base retriever:** retrieves the initial set of documents based on the query. This is similar to the retriever from the previous task.
2.   **Document compressor:** processes these documents to extract the relevant content. We use `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.


In [57]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter
from langchain.llms import OpenAI

### your code ###
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
### your code ###

Let's take a look at an example of compression retriever works.

In [58]:
print("Fist question in the set:",questions[2]['text'])
compressed_docs = compression_retriever.get_relevant_documents(questions[2]['text'])
compressed_docs

Fist question in the set: Why do more students tune into Mark's show?


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


[Document(page_content='* "more students tune into Mark\'s show"\n* "Happy Harry Hardon letters"\n* "you have a responsibility for the people who believe in you"', metadata={'source': 'local'}),
 Document(page_content='* Nora got expelled\n* Nora has been cutting lessons\n* Nora used profanity towards Creswood\n* Jan was fired\n* Jan tried to change things but forgot not to rock the boat', metadata={'source': 'local'}),
 Document(page_content='* "the kids who need the most help like those with drug problems"\n* "they don\'t go in for all that"\n* "this radio person is the whole problem"', metadata={'source': 'local'}),
 Document(page_content='* "Look the deal is I get decent grades and you guys leave me alone."\n* "I just wrote it late last night."', metadata={'source': 'local'})]

Look at the output and try out several different questions by yourself. Does the compressed output make sense?

Compare this to the previous **simple** approach. Which one, in your opinion, is better?

Finally, we use the new retriever with the Llama2 model from the previous task to create the context compressor RAG pipeline. The code below should be similiar to what you did in the previous task. Once again, make sure to turn off the `verbose` argument.

In [59]:
### your code ###
from langchain.chains import RetrievalQA

rag_compressor = RetrievalQA.from_llm(llm=llm, retriever=compression_retriever, verbose=False)
### your code ###


In [60]:
rag_compressor(questions[2]['text'])

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


{'query': "Why do more students tune into Mark's show?",
 'result': " More students tune into Mark's show because he provides a unique perspective on current events and issues that resonates with them. His show is known for being unapologetic and honest, which appeals to students who are looking for a voice that speaks their truth. Additionally, Mark's show often features guests who are experts in their fields, providing valuable insights and information that students can use to better understand the world around them."}

Now we can use the pipeline to generate answers for all the questions in our dataset. **It should take around 20 minutes on a T4 GPU on Colab.**

In [61]:
compressor_answers=[]
### your code ###
for question in tqdm(questions):
  compressor_answers.append(rag_compressor(question['text']))
### your code ###


  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


#### ${\color{red}{Comments\ 2.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 2.2. Evaluate

Since we have access to ground truth answers, we can use various evaluation metrics from the literature. In this task, we explore three metrics:


1.   **BLEU:** BLEU score stands for Bilingual Evaluation Understudy and is a precision-based metric developed
for evaluating machine translation. BLEU scores a candidate by computing the
number of n-grams in the candidate that also appear
in a reference. The n can vary, in this task we compute for n=4.
2.   **ROUGE:** ROUGE score stands for Recall-Oriented Understudy for Gisting Evaluation and is an F-measure metric designed for
evaluating translation and summarization. There are a number of variants of ROUGE.
3. **BERTScore:** BERTScore first obtains BERT representation of each word in the candidate and reference by feeding the candidate
and reference through a BERT model separately.
An alignment is then computed between candidate
and reference words by computing pairwise cosine
similarity. This alignment is then aggregated in to
precision and recall scores before being aggregated
into a (modified) F1 score that is weighted using
inverse-document-frequency values.

Luckily, Hugging Face has an implementation for all these metrics. Use the `evaluate` library to load the metrics.

Use the loaded metrics to compare the RAG pipelines from the previous subtask.

In [62]:
import evaluate
### your code ###
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
### your code ###

As seen in the previous subtask, the answers can contain multiple lines. To be able to compare the output of our systems to the gold answers, merge the multiple answers into a single string.

In [63]:
answers_merged=[]
### your code ###
answers_merged = [' '.join(entry['text'] for entry in answer) for answer in answers]
### your code ###
print(len(answers_merged))

30


Compute the BLUE score for the simple RAG and compressor RAG.

In [64]:
simple_answers_results = [answer['result'] for answer in simple_answers]
compressor_answers_results = [answer['result'] for answer in compressor_answers]

In [65]:
### your code ###
bleu_simple = bleu.compute(predictions=simple_answers_results, references=answers_merged)
bleu_compressor = bleu.compute(predictions=compressor_answers_results, references=answers_merged)
### your code ###
print("Simple system:")
print(bleu_simple)
print("Compressor:")
print(bleu_compressor)

Simple system:
{'bleu': 0.0, 'precisions': [0.10925449871465295, 0.014705882352941176, 0.001392757660167131, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.6283783783783785, 'translation_length': 778, 'reference_length': 296}
Compressor:
{'bleu': 0.0, 'precisions': [0.07709497206703911, 0.003468208092485549, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 3.0236486486486487, 'translation_length': 895, 'reference_length': 296}


What does the elements below in the output of the BLEU impelementation in Hugging Face mean? (do not copy and paste the documentation but write the implications behind each element!).



1. **precision:**
The precision scores in the array are calculated based on n-grams from 1 to 4, whereas the total number of n-grams is the foundation for calculation here. Given the 0.1 for 1-grams and even values below 1 for 2/3/4-grams the overlap to the gold-label is extremely low. Since the compressor is probably loosing information form the initially retrieved document, the overlap is even lower.
2. **brevity_penalty:**
The brevity penalty representative for the length difference between the predictiona and the gold label. It is calculated as min(1, (reference_length / translation_length)). In our scenario the value is 1, which means the prediction is longer or equal to the gold label. This counts for both generations.
3. **translation_length:**
Absolute length of words in the model prediction.
4. **reference_length:**
Absolute length of words in the gold label text.
5. **length_ratio:**
(translation_length / reference_length) In this case the prediction is way longer then the gold label resulting in a high length ratio. Both simple and compressor system generate longer outputs than the gold-label.


In [66]:
### your code ###
rouge_simple = rouge.compute(predictions=simple_answers_results, references=answers_merged)
rouge_compressor = rouge.compute(predictions=compressor_answers_results, references=answers_merged)
### your code ###
print("Simple system:")
print(rouge_simple)
print("Compressor:")
print(rouge_compressor)

Simple system:
{'rouge1': 0.12030352407284557, 'rouge2': 0.01820921985815603, 'rougeL': 0.09954794903983252, 'rougeLsum': 0.10215290791846135}
Compressor:
{'rouge1': 0.09720321512087951, 'rouge2': 0.005865594100888218, 'rougeL': 0.0787639292747971, 'rougeLsum': 0.0764287269963567}


What is the difference in variants of ROUGE (ROUGE-N, ROUGE-L, ROUGE-SUM)?

**ROUGE** assesses the likeness between the machine-generated summary and the reference summaries by analyzing shared n-grams, which are word sequences present in both the machine-generated summary and the reference summaries. The prevalent n-grams considered include unigrams, bigrams, and trigrams. The ROUGE score determines the recall of n-grams in the machine-generated summary by comparing them to the reference summaries.

**ROUGE:**
The ROUGE score is simple calculated by dividing the overlap of n-grams (of prediction and gold-label) by the total number of n-grams. The metric is an easy way for measuring the overlap between prediction and gold-label. In 

**ROUGE-N:** 
The ROUGE-N score computes precision, recall, and F1-score for n-grams with different n.
It therefore can be utalized to assess the fluency of a text.

**ROUGE-L:** 
 ROUGE-L calculates the same scores as ROUGE-N, but just uses the length of the longest subsequence between prediction and gold label.
 It then can be used to quantify the amount of word structure similarity.

**ROUGE-S:** 
ROUGE-S combines ROUGE-N and ROUGE-S by balancing them through a parameter alpha. Whereas a high alpha emphasizes ROUGE-L (the exact content as in the gold label - consecutive sequences) and a low alpha ROUGE-N (focus on the wording used). With this combination it is possible to evaluate skip-grams where a word like a color is different to the gold label.


In [67]:
import numpy as np
bertscore_simple_averaged={}
bertscore_compressor_averaged={}
### your code ###
bertscore_simple = bertscore.compute(predictions=simple_answers_results, references=answers_merged, lang="en")
bertscore_compressor = bertscore.compute(predictions=compressor_answers_results, references=answers_merged, lang="en")

print(bertscore_simple)
print(bertscore_compressor)

bertscore_simple_averaged['precision'] = sum(bertscore_simple['precision']) / len(bertscore_simple['precision'])
bertscore_simple_averaged['recall'] = sum(bertscore_simple['recall']) / len(bertscore_simple['recall'])
bertscore_simple_averaged['f1'] = sum(bertscore_simple['f1']) / len(bertscore_simple['f1'])

bertscore_compressor_averaged['precision'] = sum(bertscore_compressor['precision']) / len(bertscore_compressor['precision'])
bertscore_compressor_averaged['recall'] = sum(bertscore_compressor['recall']) / len(bertscore_compressor['recall'])
bertscore_compressor_averaged['f1'] = sum(bertscore_compressor['f1']) / len(bertscore_compressor['f1'])

### your code ###
print("Simple system:")
print(bertscore_simple_averaged)
print("Compressor:")
print(bertscore_compressor_averaged)

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.8562731742858887, 0.8667235374450684, 0.8598740696907043, 0.8245542645454407, 0.8405142426490784, 0.8333334922790527, 0.8355945348739624, 0.8550053834915161, 0.816573977470398, 0.8412203788757324, 0.8259439468383789, 0.8011640310287476, 0.8002252578735352, 0.8705782294273376, 0.8451447486877441, 0.811764657497406, 0.8150767683982849, 0.8110718727111816, 0.8330411911010742, 0.8334317803382874, 0.8422207832336426, 0.8818323612213135, 0.8445413708686829, 0.8438080549240112, 0.8212113380432129, 0.8534775376319885, 0.8533546924591064, 0.8339012861251831, 0.8466808795928955, 0.8286417126655579], 'recall': [0.8464705944061279, 0.8839300870895386, 0.8875840902328491, 0.9138057231903076, 0.8303825855255127, 0.8589107394218445, 0.8580857515335083, 0.8750464916229248, 0.8875077366828918, 0.8463096618652344, 0.8516240119934082, 0.8310544490814209, 0.8212918043136597, 0.8269176483154297, 0.8557012677192688, 0.84305340051651, 0.8306249380111694, 0.8214539289474487, 0.872717022895813

Which model works better?

Throughout all scores, es the simple system is ahead of the compressor system. As already mentioned the small score differences are most likely caused by the change in sentence structure and information loss through compression.

#### ${\color{red}{Comments\ 2.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$